# 2022 LOL WORLD CHAMPIONSHIP DA

## DATA LOAD & library load

In [121]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

from sklearn.preprocessing import MinMaxScaler

In [1]:
import pandas as pd
PATH = ""

champions = pd.read_csv(PATH + "wc_champions.csv", encoding='utf-8-sig')
players_main = pd.read_csv(PATH + "wc_players_main.csv", encoding='utf-8-sig')
players_pi = pd.read_csv(PATH + "wc_players_play_in.csv", encoding='utf-8-sig')
teams_main = pd.read_csv(PATH + "wc_teams_main.csv", encoding='utf-8-sig')
teams_pi = pd.read_csv(PATH + "wc_teams_play_in.csv", encoding='utf-8-sig')
lol2022season = pd.read_csv(PATH + "2022matchdata.csv" , encoding='utf-8-sig')
lck2022season = pd.read_csv(PATH + "lck2022.csv" , encoding='utf-8-sig')

C:\Users\Admin\AppData\Local\Temp\ipykernel_18404\2608461288.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  lol2022season = pd.read_csv(PATH + "2022matchdata.csv" , encoding='utf-8-sig')


### DATA 확인 및 결측값 확인


> 데이터
- lol2022season
    - 2022시즌 모든 lolesports 경기데이터
- cahmpions
    - 게임 내 champion 통계
- players_main
    - main stage player들의 통계
- players_pi
    - playin stage player들의 통계
- teams_main
    - main stage team들의 통계
- players_main
    - palyin stage team들의 통계

---

관심 column 선택 및 정의서

- 팀분석

    - 게임 고유 ID 매치별, 정규시즌경기 다전제 3판2선승 ->경기승리, 플레이오프 5판3선승 경기승리
    - participantid : (1,2,3...10) 선수데이터, 100블루팀 200레드팀 팀데이터
    - side : blue,red team
    - league : 리그명
    - split : 시즌(spring,summer)
    - team : 팀명
    - playoffs : playoff 유무
    - date : 경기날짜(시간 별 분석시 사용)
    - gamelength : 경기시간(강약팀구분)
    - result(승,패) 0 : 패 1 : 승
    - totalgold 총 골드획득
    - towers
    - exp


| Stats | Definitions |
|------|:-------------------------------------|
gameid	| 게임고유ID( 다전제에서 각 매치별)
participantid	| 선수,팀 데이터 구분


#게임ID #팀인지 선수인지 participantid 100 blue 200 red (1,2,3...10 선수개인) #팀  #리그 #시간 #스플릿 #플레이오프 #승패 

In [3]:
lol2022season.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149232 entries, 0 to 149231
Columns: 123 entries, gameid to opp_deathsat15
dtypes: float64(93), int64(12), object(18)
memory usage: 140.0+ MB


In [63]:
lol2022season[ (lol2022season['participantid'] == 100) | (lol2022season['participantid']==200 ) ].isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 24872 entries, 10 to 149231
Columns: 123 entries, gameid to opp_deathsat15
dtypes: float64(93), int64(12), object(18)
memory usage: 23.5+ MB


In [57]:
lol2022season.isna().sum()
#participate가 선수지표면 팀단위의 지표는 결측값이지만. 예외있음 ( 측정되지않거나 기입오류 ) 
#url은 중국공식통계 링크만 있으므로 필요하지 않음 컬럼

gameid                   0
datacompleteness         0
url                 126828
league                   0
year                     0
                     ...  
assistsat15          21828
deathsat15           21828
opp_killsat15        21828
opp_assistsat15      21828
opp_deathsat15       21828
Length: 123, dtype: int64

----

In [123]:
#추후 해보자 ( drx 선수별 지표,능력치차트 )

lck_spring_po_df = lck_spring_po[ ['gameid','playername','participantid','side','league','split','teamname','date', \
    'gamelength','result','dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore'] ]

lck_spring_po_drxplyaer_df = lck_spring_po_df[ (lck_spring_po_df['teamname'] == 'DRX') & (lck_spring_po_df['participantid'] <= 10 ) ]

lck_spring_po_drxplyaer_df.head()

,gameid,playername,participantid,side,league,split,teamname,date,gamelength,result,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
52044,ESPORTSTMNT02_2663759,Kingen,1,Blue,LCK,Spring,DRX,2022-03-24 06:05:22,2247,0,837.1963,387.6903,5873.0,7471.0,7.7437,53.0
52045,ESPORTSTMNT02_2663759,Pyosik,2,Blue,LCK,Spring,DRX,2022-03-24 06:05:22,2247,0,206.9693,651.1081,5592.0,5445.0,5.6609,72.0
52046,ESPORTSTMNT02_2663759,ZEKA,3,Blue,LCK,Spring,DRX,2022-03-24 06:05:22,2247,0,340.2670,509.5327,5807.0,7161.0,8.2510,36.0
52047,ESPORTSTMNT02_2663759,Deft,4,Blue,LCK,Spring,DRX,2022-03-24 06:05:22,2247,0,719.4927,520.4806,5071.0,6108.0,9.1055,50.0
52048,ESPORTSTMNT02_2663759,BeryL,5,Blue,LCK,Spring,DRX,2022-03-24 06:05:22,2247,0,82.8037,486.8358,3123.0,3018.0,1.3885,106.0


In [127]:
lck_spring_po_drxplayer_score = lck_spring_po_drxplyaer_df[ ['playername','teamname','dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore'] ]
lck_spring_po_drxplayer_score.head()

,playername,teamname,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
52044,Kingen,DRX,837.1963,387.6903,5873.0,7471.0,7.7437,53.0
52045,Pyosik,DRX,206.9693,651.1081,5592.0,5445.0,5.6609,72.0
52046,ZEKA,DRX,340.2670,509.5327,5807.0,7161.0,8.2510,36.0
52047,Deft,DRX,719.4927,520.4806,5071.0,6108.0,9.1055,50.0
52048,BeryL,DRX,82.8037,486.8358,3123.0,3018.0,1.3885,106.0


In [129]:
X = lck_spring_po_drxplayer_score[['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']]

In [130]:
scaler = MinMaxScaler(feature_range=(0,5)) ## 각 칼럼 데이터 값을 0~6 범위로 변환
scaler.fit(X) ## 각 칼럼 데이터마다 변환할 함수 생
transformed_X = scaler.transform(X) ## fit에서 만들어진 함수를 실제로 데이터에 적용
transformed_X

array([[3.83221559e+00, 1.00744343e+00, 5.00000000e+00, 4.81509516e+00,
        3.90702800e+00, 1.49038462e+00],
       [7.21579819e-01, 3.22792786e+00, 4.48909091e+00, 2.62435121e+00,
        2.69266649e+00, 2.40384615e+00],
       [1.37950243e+00, 2.03451583e+00, 4.88000000e+00, 4.47988754e+00,
        4.20280560e+00, 6.73076923e-01],
       [3.25126138e+00, 2.12680132e+00, 3.54181818e+00, 3.34126298e+00,
        4.70101566e+00, 1.34615385e+00],
       [1.08730868e-01, 1.84319196e+00, 0.00000000e+00, 0.00000000e+00,
        2.01674499e-01, 4.03846154e+00],
       [2.78841298e+00, 2.66164481e-01, 2.70000000e+00, 4.62694637e+00,
        3.77759250e+00, 1.73076923e+00],
       [1.29323362e+00, 3.86724508e+00, 2.63454545e+00, 2.35618512e+00,
        2.63418730e+00, 1.49038462e+00],
       [2.57007268e+00, 1.28042655e+00, 4.33818182e+00, 4.19117647e+00,
        4.16840608e+00, 2.06730769e+00],
       [5.00000000e+00, 1.64787924e+00, 4.22363636e+00, 2.24913495e+00,
        4.50126520e+00, 

In [133]:
lck_spring_po_drxplayer_score[['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']] = transformed_X
lck_spring_po_drxplayer_score.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_18404\4169893245.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,playername,teamname,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
52044,Kingen,DRX,3.832216,1.007443,5.000000,4.815095,3.907028,1.490385
52045,Pyosik,DRX,0.721580,3.227928,4.489091,2.624351,2.692666,2.403846
52046,ZEKA,DRX,1.379502,2.034516,4.880000,4.479888,4.202806,0.673077
52047,Deft,DRX,3.251261,2.126801,3.541818,3.341263,4.701016,1.346154
52048,BeryL,DRX,0.108731,1.843192,0.000000,0.000000,0.201674,4.038462


In [139]:
drx_spring_po_score = lck_spring_po_drxplayer_score.groupby('playername')\
    [['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']].mean()

drx_spring_po_score

,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
playername,,,,,,
BeryL,0.263728,1.176329,0.601455,0.839533,0.114335,4.346154
Deft,3.254800,1.043306,3.854909,2.694420,4.769360,1.221154
Kingen,2.085118,1.981356,3.345455,4.585424,3.466096,1.259615
Pyosik,0.903991,3.660601,3.550909,2.553633,2.560222,1.875000
ZEKA,2.028368,2.138103,4.290909,4.619161,4.028569,1.298077


In [149]:
drx_spring_po_score.iloc[0].index

Index(['dpm', 'damagetakenperminute', 'goldat15', 'xpat15', 'cspm',
       'visionscore'],
      dtype='object')

In [151]:
import plotly.express as px

fig = px.line_polar(r=drx_spring_po_score.iloc[1].values , theta=drx_spring_po_score.iloc[1].index, line_close=True)

fig.show()

---

### lck리그
#### split 별 분류
##### playoff 유무

In [12]:
lck = lol2022season[ lol2022season['league'] == 'LCK' ]
lck_spring = lck[ (lck['split'] == 'Spring') & (lck['playoffs']==0) ]
lck_summer = lck[ (lck['split'] == 'Summer') & (lck['playoffs']==0) ]
lck_spring_po = lck[ (lck['split'] == 'Spring') & (lck['playoffs']==1) ]
lck_summer_po = lck[ (lck['split'] == 'Summer') & (lck['playoffs']==1) ]

In [29]:
lck_spring_df = lck_spring[ ['gameid','participantid','side','league','split','teamname','date', \
    'gamelength','result','totalgold','golddiffat15','towers'] ]
lck_spring_df

,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
1440,ESPORTSTMNT01_2700815,1,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,0,13189,-202.0,NaN
1441,ESPORTSTMNT01_2700815,2,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,0,10792,240.0,NaN
1442,ESPORTSTMNT01_2700815,3,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,0,14226,438.0,NaN
1443,ESPORTSTMNT01_2700815,4,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,0,16249,2849.0,NaN
1444,ESPORTSTMNT01_2700815,5,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,0,9291,1432.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
49051,ESPORTSTMNT02_2672758,8,Red,LCK,Spring,Fredit BRION,2022-03-20 10:37:02,2499,0,14050,-944.0,NaN
49052,ESPORTSTMNT02_2672758,9,Red,LCK,Spring,Fredit BRION,2022-03-20 10:37:02,2499,0,18472,686.0,NaN
49053,ESPORTSTMNT02_2672758,10,Red,LCK,Spring,Fredit BRION,2022-03-20 10:37:02,2499,0,8375,-1017.0,NaN
49054,ESPORTSTMNT02_2672758,100,Blue,LCK,Spring,Kwangdong Freecs,2022-03-20 10:37:02,2499,1,71556,703.0,11.0


In [91]:
lck_spring_team_df = lck_spring_df[ (lck_spring_df['participantid'] == 100) | (lck_spring_df['participantid'] == 200 ) ]

lck_spring_team_df['result'] = np.where( lck_spring_team_df['result'] == 0 , 'lose', 'win')

lck_spring_team_df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_18404\3910339335.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
1450,ESPORTSTMNT01_2700815,100,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,lose,63747,4757.0,8.0
1451,ESPORTSTMNT01_2700815,200,Red,LCK,Spring,Liiv SANDBOX,2022-01-12 06:20:03,2195,win,67669,-4757.0,9.0
1462,ESPORTSTMNT01_2690695,100,Blue,LCK,Spring,DRX,2022-01-12 09:02:13,2070,lose,60674,-1045.0,3.0
1463,ESPORTSTMNT01_2690695,200,Red,LCK,Spring,Liiv SANDBOX,2022-01-12 09:02:13,2070,win,67152,1045.0,9.0
1486,ESPORTSTMNT01_2690705,100,Blue,LCK,Spring,T1,2022-01-12 10:07:10,2233,win,66455,1309.0,7.0


In [67]:
lck_spring_team_df.isna().sum() #결측값확인

gameid           0
participantid    0
side             0
league           0
split            0
teamname         0
date             0
gamelength       0
result           0
totalgold        0
golddiffat15     0
towers           0
dtype: int64

In [88]:
px.box( y= lck_spring_team_df['totalgold'] )
# 게임시간이 매우 긴 경기 totalgold 비약적으로 증가
# 따라서 15diff 지표로 확인하는것이 적절

In [89]:
lck_spring_team_df[ lck_spring_team_df['totalgold'] >= 90000 ]

,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
10750,ESPORTSTMNT02_2552430,100,Blue,LCK,Spring,DWG KIA,2022-01-30 06:56:25,3326,1,100872,448.0,10.0
10751,ESPORTSTMNT02_2552430,200,Red,LCK,Spring,T1,2022-01-30 06:56:25,3326,0,93420,-448.0,7.0
18922,ESPORTSTMNT02_2578644,100,Blue,LCK,Spring,T1,2022-02-12 06:05:04,2746,1,90436,-567.0,8.0


In [54]:
lck_spring_team_df['teamname'].unique()

array(['DRX', 'Liiv SANDBOX', 'T1', 'Kwangdong Freecs',
       'Nongshim RedForce', 'Hanwha Life Esports', 'KT Rolster',
       'DWG KIA', 'Gen.G', 'Fredit BRION'], dtype=object)

In [92]:
lck_spring_team_df.describe()

,participantid,gamelength,totalgold,golddiffat15,towers
count,424.000000,424.000000,424.000000,424.000000,424.000000
mean,150.000000,2057.292453,60717.971698,0.000000,6.202830
std,50.059067,378.859068,11860.644727,2590.450046,3.571809
min,100.000000,1145.000000,28982.000000,-10037.000000,0.000000
25%,100.000000,1780.000000,52468.250000,-1474.500000,3.000000
50%,150.000000,2030.500000,60665.500000,0.000000,7.000000
75%,200.000000,2316.250000,68100.000000,1474.500000,9.000000
max,200.000000,3326.000000,100872.000000,10037.000000,11.000000


> ### DRX 경기만 확인

In [42]:
lck_spring_drx_df = lck_spring_df[ (lck_spring_df['teamname'] == 'DRX') & ( (lck_spring_df['participantid'] == 100) \
    | (lck_spring_df['participantid'] == 200 ) ) ]

lck_spring_drx_df['result'] = np.where( lck_spring_drx_df['result'] == 0 , 'lose', 'win')

lck_spring_drx_df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_18404\2011406639.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lck_spring_drx_df['result'] = np.where( lck_spring_drx_df['result'] == 0 , 'lose', 'win')


,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
1450,ESPORTSTMNT01_2700815,100,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,lose,63747,4757.0,8.0
1462,ESPORTSTMNT01_2690695,100,Blue,LCK,Spring,DRX,2022-01-12 09:02:13,2070,lose,60674,-1045.0,3.0
2386,ESPORTSTMNT01_2691507,100,Blue,LCK,Spring,DRX,2022-01-14 07:15:41,1778,lose,45102,-787.0,4.0
2423,ESPORTSTMNT01_2691515,200,Red,LCK,Spring,DRX,2022-01-14 08:42:06,1556,lose,40047,-6103.0,0.0
4978,ESPORTSTMNT01_2704285,100,Blue,LCK,Spring,DRX,2022-01-20 06:11:55,1676,lose,46838,-2197.0,1.0


In [43]:
lck_spring_drx_df.describe() #drx 지표통계확인

,participantid,gamelength,totalgold,golddiffat15,towers
count,41.000000,41.000000,41.000000,41.000000,41.000000
mean,148.780488,2066.317073,60768.268293,20.097561,6.439024
std,50.606083,380.109355,12588.163567,2302.973337,3.721886
min,100.000000,1537.000000,39051.000000,-6103.000000,0.000000
25%,100.000000,1778.000000,53257.000000,-1469.000000,3.000000
50%,100.000000,1967.000000,60094.000000,-378.000000,8.000000
75%,200.000000,2367.000000,69709.000000,1079.000000,10.000000
max,200.000000,2887.000000,87529.000000,4904.000000,11.000000


In [52]:
lck_spring_drx_df['totalgold'].mean() #drx 지표통계확인

60768.26829268293

In [98]:
# totalgold 평균 지표
lck_spring_team_scored = lck_spring_team_df.groupby('teamname')[ ['towers','gamelength'] ].mean()
lck_spring_team_scored

,towers,gamelength
teamname,,
DRX,6.439024,2066.317073
DWG KIA,7.166667,2143.571429
Fredit BRION,6.000000,2133.750000
Gen.G,7.750000,1992.350000
Hanwha Life Esports,4.465116,2093.116279
KT Rolster,5.418605,2015.186047
Kwangdong Freecs,6.571429,2101.309524
Liiv SANDBOX,4.121951,1906.585366
Nongshim RedForce,5.466667,2094.955556


In [107]:
fig = go.Figure()

fig.add_trace(go.Bar(x=lck_spring_team_scored.index , y=lck_spring_team_scored['gamelength']) )
fig.update_yaxes(range=[1900,2200])

fig.show()

In [101]:
fig = go.Figure()

fig.add_trace(go.Bar(x=lck_spring_team_scored.index , y=lck_spring_team_scored['towers']) )
fig.update_yaxes(range=[3, 9])

fig.show()

In [84]:
fig = go.Figure()

fig.add_trace(go.Bar(x=lck_spring_team_total_gold.index , y=lck_spring_team_total_gold.values) )
fig.update_yaxes(range=[58000, 65000])

fig.show()

In [74]:
# 15분 gold차이 평균 지표
lck_spring_team_golddiff_mean = lck_spring_team_df.groupby('teamname')['golddiffat15'].mean()
lck_spring_team_golddiff_mean

teamname
DRX                      20.097561
DWG KIA                 722.666667
Fredit BRION           -586.863636
Gen.G                   619.775000
Hanwha Life Esports    -519.813953
KT Rolster             -221.209302
Kwangdong Freecs       -221.333333
Liiv SANDBOX          -1731.609756
Nongshim RedForce        11.177778
T1                     1895.534884
Name: golddiffat15, dtype: float64

In [75]:
fig = go.Figure()

fig.add_trace(go.Bar(x=lck_spring_team_golddiff_mean.index , y=lck_spring_team_golddiff_mean.values) )

fig.show()

In [ ]:
# 15분 gold차이 평균 지표
lck_spring_team_golddiff_mean = lck_spring_team_df.groupby('teamname')['tower'].mean()
lck_spring_team_golddiff_mean

In [ ]:
def lck_spring_team_goldmean_cal:
    for i in range


In [2]:
# 승패 확인을 위한 전처리
lol2022season.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0
1,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,119.0,-475.0,153.0,1.0,0.0,3.0,0.0,3.0,3.0,2.0
3,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,149.0,-793.0,-1343.0,-34.0,2.0,1.0,2.0,3.0,3.0,0.0
4,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,21.0,443.0,-497.0,7.0,1.0,2.0,2.0,0.0,6.0,2.0
